<a href="https://colab.research.google.com/github/enricolopesmarques/Projeto_Imersao_IA_Alura/blob/main/IA_Assistente_Compras_Multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U ipywidgets
!pip install -q -U google.generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00


In [2]:
# Importa as bibliotecas utilizadas no programa
import ipywidgets as widgets # --> Inerface gráfica no terminal
import google.generativeai as genai # --> Importa o Gemini e cria apelido que facilita a digitação
import pathlib # --> Otimiza a funcionalidade de trabalhar com caminhos de arquivos
import textwrap # --> Auxilia na formatação de textos
import PIL.Image # --> Habilita a funcionalidade de trabalhar com imagens
from IPython.display import display
from google.colab import files # --> Habilita a funcionalidade de importação de arquivos

# Responsável por associar a chave API de forma segura pelo Colab Secrets (COLE A SUA API KEY NO CÓDIGO ABAIXO!)
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

from IPython.display import display
from IPython.display import Markdown

#Atibui a variável 'model' o modelo de IA escolhido
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest')

# Apresentação do sistema e explicações iniciais
print('-='*45)
print(f'{" ASSISTENTE DE COMPRAS DE SUPERMERCADO ":-^90}')
print('-='*45)
print('Bem vindo ao Assistente de Compras Inteligente!')
print()
print('''Com esta ferramenta, você pode criar listas de compras personalizadas em segundos.
Nosso sistema inteligente, integrado com a IA do Google Gemini, gera uma lista completa:
- Segmentada por categoria para facilitar a localização dos produtos no supermercado.
- Quantidades adequadas para evitar desperdícios
- Consideração das suas restrições alimentares e preferências''')
print()
print('Para isso vou precisar apenas de algumas informações.')
print('-'*90)


def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Função para upload da imagem
def upload_imagem(botao):
  global imagem_carregada # Declarar a variável como global
  uploaded = files.upload()
  for nome_arquivo in uploaded.keys():
    print()
    print(f"Imagem '{nome_arquivo}' carregada com sucesso!")
    # Abrir a imagem e armazenar na variável global
    imagem_carregada = PIL.Image.open(nome_arquivo)
    print()
    print('Clique para gerar a lista de Compras: ')
    display(botao_enviar_com_imagem)

def prompt_com_imagem(botao):
  print()
  print('Aguarde alguns segundos, estou analisando as suas informações e gerando a melhor lista de compras pra você...')
  print()

  # coleta os dados dos widgets
  num_pessoas = num_pessoas_widget.value
  duracao = duracao_widget.value
  restricoes = restricoes_widget.value
  alergia = alergia_alimentos
  refeicoes = refeicoes_widget.value

  prompt = (f'''Analise a imagem, liste os produtos e quantidade. Considerandos os produtos que identificar como o meu estoque,
Gere uma lista de compras para uma família de {num_pessoas} pessoas para uma {duracao}
desconsiderando os produtos que eu já tenho em estoque (imagem) para evitar desperdícios.

A lista deve considerar também os critérios abaixo

Restrições alimentares: {restricoes}
Alergia: {alergia}

Hábitos:
* Refeições em casa: {refeicoes}

Formato da lista:
* Agrupar itens por categoria (frutas, verduras, laticínios, etc.)
* Especificar quantidades para cada item.''')

  # Preenche o prompt com os inputs do usuário
  response = model.generate_content([imagem_carregada, prompt])

  #Exibe a lista gerada pela IA
  print()
  display(to_markdown(response.text))

# Função que é chamada quando o botão de Gerar Lista é clicado.
def enviar_dados(sender):
  print()
  print('Aguarde alguns segundos, estou analisando as suas informações e gerando a melhor lista de compras pra você...')
  print()

  # coleta os dados dos widgets
  num_pessoas = num_pessoas_widget.value
  duracao = duracao_widget.value
  restricoes = restricoes_widget.value
  alergia = alergia_alimentos
  refeicoes = refeicoes_widget.value

  prompt = (f'''Gere uma lista de compras para uma família de {num_pessoas} pessoas para uma {duracao}

Restrições alimentares: {restricoes}
Alergia: {alergia}

Hábitos:
* Refeições em casa: {refeicoes}

Formato da lista:
* Agrupar itens por categoria (frutas, verduras, laticínios, etc.)
* Especificar quantidades para cada item.''')

  # Preenche o prompt com os inputs do usuário
  response = model.generate_content(prompt)

  #Exibe a lista gerada pela IA
  print()
  display(to_markdown(response.text))



# Configuração dos Widgets que coletam informações
num_pessoas_widget = widgets.IntSlider(min=1, max=10, description="Nº Pessoas:")
duracao_widget = widgets.Dropdown(
    options=["Compra do mês", "Compra quinzenal", "Compra da semana"],
    description="Duração:")

restricoes_widget = widgets.SelectMultiple(
    options=["Nenhuma restrição","Sem glúten", "Sem lactose", "Vegetariano", "Vegano"],
    description="Restrições:")

refeicoes_widget = widgets.SelectMultiple(
    options=["Café da manhã", "Almoço","Jantar"],
    description="Refeições em Casa:",)

alergia_widget = widgets.Dropdown(
    options=["Compra do mês", "Compra quinzenal", "Compra da semana"],
    description="Duração:")

# Configuração do Botão que coleta as informações e armazena nas variáveis
botao_enviar = widgets.Button(description="Gerar Lista")
botao_enviar.on_click(enviar_dados)

# Criar botão para upload
botao_upload = widgets.Button(description="Selecionar Imagem")
botao_upload.on_click(upload_imagem)

# Botão que envia a imagem para o Gemini
botao_enviar_com_imagem = widgets.Button(description="Gerar Lista")
botao_enviar_com_imagem.on_click(prompt_com_imagem)

# Exibição dos widgets
print()
print('A compra será para quantas pessoas?')
display(num_pessoas_widget)
print()
print('Para qual período você quer realizar a sua compra?')
display(duracao_widget)
print()
print('Você possui alguma restrição alimentar? (segure CTRL E CLIQUE para selecionar mais de uma opção)')
display(restricoes_widget)
print()
print('Quantas refeições você faz em casa? (segure CTRL E CLIQUE para selecionar mais de uma opção)')
display(refeicoes_widget)
print()

alergia_input = str(input(f"{'Você é alérgico a algum alimento? [S/N]: ':<10}")).strip().upper()[0]
while alergia_input not in 'SN':
  print()
  alergia_input = str(input(f"{'Valor inválido, digite [S ou N]: ':<10}")).strip().upper()[0]
if alergia_input == 'S':
  print()
  alergia_alimentos = str(input(f"{'A quais alimentos você é alérgico? (adicione uma lista de alimentos separada por vírgula): ':<10}"))
if alergia_input == 'N':
  alergia_alimentos = 'Nenhuma alergia a alimentos específicos'
print('-'*90)
print()
print('''Eu consigo analisar uma imagem da sua dispensa ou geladeira e desconsiderar os produtos que você já tem em casa na geração da lista de compras.''')
print()
usa_imagem = str(input('Deseja usar essa funcionalidade? [S/N]: ')).strip().upper()[0]
while usa_imagem not in 'SN':
  usa_imagem == str(input('Valor inválido, escolha uma opção válida [S ou N]: ')).strip().upper()[0]
if usa_imagem == 'S':
  print()
  print('-'*90)
  print('Para que eu consiga analisar com mais precisão faça o upload de uma imagem com boa iluminação e que apareça todos os produtos de forma clara.')
  print()
  print('Clique no botão abaixo para fazer o upload')
  print()
  display(botao_upload)
if usa_imagem == 'N':
  # Exibição do botão que Gera a Lista
  print()
  print('-'*90)
  display(botao_enviar)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
------------------------- ASSISTENTE DE COMPRAS DE SUPERMERCADO --------------------------
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Bem vindo ao Assistente de Compras Inteligente!

Com esta ferramenta, você pode criar listas de compras personalizadas em segundos.
Nosso sistema inteligente, integrado com a IA do Google Gemini, gera uma lista completa:
- Segmentada por categoria para facilitar a localização dos produtos no supermercado.
- Quantidades adequadas para evitar desperdícios
- Consideração das suas restrições alimentares e preferências

Para isso vou precisar apenas de algumas informações.
------------------------------------------------------------------------------------------

A compra será para quantas pessoas?


IntSlider(value=1, description='Nº Pessoas:', max=10, min=1)


Para qual período você quer realizar a sua compra?


Dropdown(description='Duração:', options=('Compra do mês', 'Compra quinzenal', 'Compra da semana'), value='Com…


Você possui alguma restrição alimentar? (segure CTRL E CLIQUE para selecionar mais de uma opção)


SelectMultiple(description='Restrições:', options=('Nenhuma restrição', 'Sem glúten', 'Sem lactose', 'Vegetari…


Quantas refeições você faz em casa? (segure CTRL E CLIQUE para selecionar mais de uma opção)


SelectMultiple(description='Refeições em Casa:', options=('Café da manhã', 'Almoço', 'Jantar'), value=())


Você é alérgico a algum alimento? [S/N]: N
------------------------------------------------------------------------------------------

Eu consigo analisar uma imagem da sua dispensa ou geladeira e desconsiderar os produtos que você já tem em casa na geração da lista de compras.

Deseja usar essa funcionalidade? [S/N]: S

------------------------------------------------------------------------------------------
Para que eu consiga analisar com mais precisão faça o upload de uma imagem com boa iluminação e que apareça todos os produtos de forma clara.

Clique no botão abaixo para fazer o upload



Button(description='Selecionar Imagem', style=ButtonStyle())